In [1]:
from pymystem3 import Mystem
import pymorphy2
import networkx as nx
from os import listdir
from numpy import arange, log, sqrt

In [2]:
# морфологический анализатор Mystem
mystem = Mystem()

# морфологический анализатор PyMorphy
pymorphy = pymorphy2.MorphAnalyzer()

In [3]:
# директория хранения текстовых файлов для тестирования
text_path = 'texts'

# считывание текстовых файлов из заданной директории в список текстов
texts = [open(text_path + '/' + filename, 'r').read() for filename in listdir(text_path) if not filename.startswith('.')]

In [4]:
# функция конкатенации списков
def concat(lists):
    result = []
    for lst in lists:
        for elem in lst:
            result.append(elem)
    return result

# предикат: является ли слово разделителем
def is_separator(word):
    return word.get('analysis') == None

# получить текст слова
def get_text(word):
    return word['text']

# извлечение из текста множества всех разделителей
separators = set(concat([list(map(get_text, filter(is_separator, mystem.analyze(text)))) for text in texts]))

In [5]:
# посмотрим на них
separators

{'\n',
 ' ',
 ' \n',
 ' "',
 ' (',
 ' «',
 ' – ',
 ' — ',
 ' “',
 '!',
 '! ',
 '!" ',
 '!) ',
 '"',
 '" ',
 '" (',
 '" — ',
 '", ',
 '#',
 ') – ',
 '), ',
 ', ',
 '-',
 '.',
 '. ',
 '... ',
 '/',
 '10',
 '11',
 '2030',
 '22',
 '26',
 '30',
 '90',
 ': ',
 ': "',
 ': «',
 '; ',
 '?',
 '? ',
 '?" ',
 '«',
 '»',
 '» ',
 '», – ',
 '“',
 '”',
 '” ',
 '” — '}

In [6]:
# элементы, ошибочно выбранные разделителями (числа)
exclude_separators = set(filter(lambda s: s.isdigit(), separators))

# удаление из множества разделителей ошибочных
separators -= exclude_separators

# разделители, завершающие предложения
end_separators = set(['!', '.', '. ', '?', '? ', '…', '… '])

In [7]:
# списочное представление текстов: [[String]]
texts_list = []

# парсинг на предложения
for text in texts:
    words = list(map(get_text, mystem.analyze(text)))
    
    sentences = []
    sentence = []
    i = 0
    while i < len(words):
        word = words[i]
        sentence.append(word)
        if word in end_separators:
            while i+1 < len(words) and words[i+1] in separators:
                i += 1
                word = words[i]
                sentence.append(word)
                continue
            sentences.append(sentence)
            sentence = []
        i += 1
    
    texts_list.append(sentences)

In [8]:
# получение леммы (нормальной формы) слова
def normal_form(word):
    return pymorphy.normal_forms(word)[0]

# получение множества слов из текста с их частотами
def words_dict(text, one_sentence=False):
    words = dict()
    if one_sentence:
        for word in text:
            if word not in separators:
                lemma = normal_form(word)
                if lemma in words:
                    words[lemma] += 1
                else:
                    words[lemma] = 1
    else:
        for sentence in text:
            for word in sentence:
                if word not in separators:
                    lemma = normal_form(word)
                    if lemma in words:
                        words[lemma] += 1
                    else:
                        words[lemma] = 1
    return words

# индекс максимального значения в списке
def i_max(a):
    if len(a) == 0:
        return -1
    
    k = 0
    m = a[0]
    for i in range(1, len(a)):
        if a[i] > m:
            m = a[i]
            k = i
    return k

# получение единой строки из списка предложений (списка списка слов)
def to_string(text):
    res = []
    for sentence in text:
        s = ''
        for word in sentence:
            s += word
        if s[-1] == ' ':
            s = s[:-1]
        res.append(s)
    return ' '.join(res)

# первый элемент пары
def fst(x):
    return x[0]

# второй элемент пары
def snd(x):
    return x[1]

In [9]:
# получение аннотации текста с заданной степенью сжатия методом SumBasic
def annotation_sumbasic(text, alpha):
    # список доступных для обработки (еще не включенных в аннотацию) предложений
    available_sentences = text.copy()
    
    # число предложений в аннотации
    annotation_size = int(alpha * len(available_sentences))
    
    # аннотация
    annotation = []
    
    for _ in range(annotation_size):
        # множество (словарь) всех слов текста с частотой их встречания в тексте
        words = words_dict(available_sentences)
        
        # значения значимости предложений
        scores = []
        for sentence in available_sentences:
            # значение значимости текущего предложения
            score = 0
            
            # множество (словарь) всех слов предложения с частотой их встречания в предложении
            sentence_words = words_dict(sentence, one_sentence=True)
            for lemma, value in sentence_words.items():
                score += value / words[lemma]
            score /= len(sentence_words)
            scores.append(score)
        
        # выбор самого значимого предложения
        idx = i_max(scores)
        
        annotation.append((idx, available_sentences[idx]))
        available_sentences.pop(idx)
    
    # перевод внутреннего представления аннотации в строку
    annotation = to_string(map(snd, sorted(annotation, key=fst)))
    
    return annotation

In [10]:
def tf(t, d):
    return d[t] / sum(d.values())

# преобразование списка слов предложений в список лемм предложений
def to_collection(text):
    collection = []
    for sentence in text:
        lemmas = set()
        for word in sentence:
            if word not in separators:
                lemma = normal_form(word)
                lemmas.add(lemma)
        collection.append(lemmas)
    return collection

def idf(t, D):
    count_d = len(list(filter(lambda d: t in d, D)))
    return log(len(D)/count_d)

# получение аннотации текста с заданной степенью сжатия методом tf-idf
def annotation_tf_idf(text, alpha):
    # список доступных для обработки (еще не включенных в аннотацию) предложений
    available_sentences = text.copy()
    
    # число предложений в аннотации
    annotation_size = int(alpha * len(available_sentences))
    
    # аннотация
    annotation = []
    
    # множество (словарь) всех слов текста с частотой их встречания в тексте
    words = words_dict(available_sentences)
    
    # создание коллекции лемм для предложений обрабатываемого текста
    collection = to_collection(text)
    
    # значения значимости предложений
    scores = []
    for sentence in available_sentences:
        score = 0

        # множество (словарь) всех слов предложения с частотой их встречания в предложении
        sentence_words = words_dict(sentence, one_sentence=True)
        for lemma, value in sentence_words.items():
            score += tf(lemma, sentence_words) * idf(lemma, collection)
        score /= len(sentence_words)
        scores.append(score)
    
    for _ in range(annotation_size):
        # выбор самого значимого предложения
        idx = i_max(scores)
        scores.pop(idx)
        
        annotation.append((idx, available_sentences[idx]))
        available_sentences.pop(idx)
    
    # перевод внутреннего представления аннотации в строку
    annotation = to_string(map(snd, sorted(annotation, key=fst)))
    
    return annotation

In [11]:
# создание вектора всех лемм текста
def template_vector(text):
    vec = set()
    for sentence in text:
        for word in sentence:
            if word not in separators:
                lemma = normal_form(word)
                vec.add(lemma)
    return list(vec)

# вычисление евклидовой нормы вектора
def norm(vec):
    val = 0
    for v in vec:
        val += v ** 2
    return sqrt(val)

# схожесть векторов по косинусной мере
def similarity(v1, v2):
    res = 0
    for i in range(len(v1)):
        res += v1[i] * v2[i]
    res /= norm(v1) * norm(v2)
    return res

# создание вектора предложения
def vector(sentence, template):
    res = {}
    for lemma in template:
        res[lemma] = 0
    for word in sentence:
        if word not in separators:
            lemma = normal_form(word)
            res[lemma] += 1
    return list(res.values())

# создание графа предложений
def graph(text, beta):
    G = nx.Graph()
    vectors = []
    template = template_vector(text)
    
    for i in range(len(text)):
        G.add_node(i)
        vectors.append(vector(text[i], template))
    
    for i in range(len(text) - 1):
        for j in range(i+1, len(text)):
            if similarity(vectors[i], vectors[j]) >= beta:
                G.add_edge(i, j)
    
    return G

# получение аннотации текста с заданной степенью сжатия
# и заданным порогом для ребер методом построения графа
def annotation_graph(text, alpha, beta):
    # список доступных для обработки (еще не включенных в аннотацию) предложений
    available_sentences = text.copy()
    
    # число предложений в аннотации
    annotation_size = int(alpha * len(available_sentences))
    
    # аннотация
    annotation = []
    
    # множество (словарь) всех слов текста с частотой их встречания в тексте
    words = words_dict(available_sentences)
    
    # создания графа предложений
    G = graph(text, beta)
    
    # значения значимости предложений
    # вычисление значений центральности для каждой из вершин по ее степени
    scores = list(nx.degree_centrality(G).values())
    
    for _ in range(annotation_size):
        # выбор самого значимого предложения
        idx = i_max(scores)
        scores.pop(idx)
        
        annotation.append((idx, available_sentences[idx]))
        available_sentences.pop(idx)
    
    # перевод внутреннего представления аннотации в строку
    annotation = to_string(map(snd, sorted(annotation, key=fst)))
    
    return annotation

In [12]:
for i, text in enumerate(texts_list):
    for alpha in arange(0.2, 0.41, 0.1):
        print('SumBasic со степенью сжатия {0:.0%} для {1}-го текста:'.format(alpha, i + 1))
        print('*' * 100)
        print(annotation_sumbasic(text, alpha))
        print('*' * 100, end='\n\n')
        
        print('TF-IDF со степенью сжатия {0:.0%} для {1}-го текста:'.format(alpha, i + 1))
        print('*' * 100)
        print(annotation_tf_idf(text, alpha))
        print('*' * 100, end='\n\n')
        
        for beta in arange(0.2, 0.41, 0.1):
            print('Граф со степенью сжатия {0:.0%} и заданным порогом {1:0.1f} для {2}-го текста:'.format(alpha, beta, i + 1))
            print('*' * 100)
            print(annotation_graph(text, alpha, beta))
            print('*' * 100, end='\n\n')

SumBasic со степенью сжатия 20% для 1-го текста:
****************************************************************************************************
Во-первых, в Москве существует огромный рэп-фестиваль с бесплатным входом и интересным лайн-апом: я прекрасно помню себя нищим студентом, воровавшим яйца в магазине через дорогу от общаги (#непытайтесьповторитьэтодома), чтоб не отъехать от голода, и я вам честно скажу, что бесплатный фестиваль с участием твоих любимых артистов – ЭТО КРУТО. 

 Много людей – много проблем. Дурак дураку говорит: «Ты – дурак», – вот так начинается любая из драк». 

 Всё устроено несколько сложнее.
****************************************************************************************************

TF-IDF со степенью сжатия 20% для 1-го текста:
****************************************************************************************************
Дальше начинаются нюансы. Много людей – много проблем. Это риск. Надеюсь на понимание.
*******************************

Возможно, всё было вообще не так. Дальше начинаются нюансы. Много людей – много проблем. Это риск. Это постановление свыше. А дальше начинается замес. « Всё устроено несколько сложнее. Надеюсь на понимание.
****************************************************************************************************

Граф со степенью сжатия 40% и заданным порогом 0.2 для 1-го текста:
****************************************************************************************************
Полной картины я по-прежнему не имею, но, в общих чертах, вроде бы ситуация выглядит так: в какой-то момент количество посетителей мероприятия превысило допустимые пределы, и охрана/полиция/Росгвардия перестали впускать вновь прибывших на территорию; это, в свою очередь, вроде как вызвало неадекватную реакцию части посетителей, и некие «пьяные школьники» начали провоцировать силовиков. Мне доводилось провоцировать силовиков – и я вам по своему опыту скажу, что это всегда заканчивается плохо. Это не значит, что силови

Как и все последующие действия: выйдем в поле против огромной армии зомби, которые будут просто бежать на нас и давить количеством — сделано; зажжем колья вокруг стен, которые мертвецы просто завалят телами — сделано; встанем на крепостной стене, на которую зомби будут лезть, как в "Войне миров Z" — сделано; знаем, что Король Ночи затем поднимет наших павших и обратит их против нас, какой тут план? — И вот в начале битвы Бран закатил глаза, отправился летать в вороне иии... Может он разведал какую-то ключевую информацию и помог защитникам Винтерфелла? Или пока все дрались, привел в действие какие-то могучие сверхъестественные силы? Окей!

 Сначала на шахматную доску возвращается запыленная фигура Мелисандры. А затем Мелисандра замотивирует поплывшую Арью, напомнив свою реплику из третьего сезона: тогда она напророчила девчонке, что та "закроет еще много глаз: карих, зеленых, голубых" (смекаешь, Арья! Голубых глаз!) И отправит ее на подвиг, для бодрости напутствуя еще и словами учителя 

Но это же очень тупо! 

 А зачем они так с Браном? Смотрите, мы имеем существо с экстраординарными способностями, по сути, комиксного супергероя. Или пока все дрались, привел в действие какие-то могучие сверхъестественные силы? Сначала на шахматную доску возвращается запыленная фигура Мелисандры. Окей!

 Неплохо увязали, да и сцена, где Арья прилетела — она же прилетела, да? Но и это еще не все. Потеряв "ходоков", сериал чуть ли не потерял смысл. Но!

 Гипотеза — даже после этой нелепейшей серии "Игра престолов" сохранит интерес. Переходим в режим ностальгии. 

 Он поднимает мертвецов. Конец списка. 


****************************************************************************************************

Граф со степенью сжатия 40% и заданным порогом 0.2 для 2-го текста:
****************************************************************************************************
Понятно, что атака дотракийцев с огненными саблями и под градом чертящих ночное небо огненных ядер, а особенно то, как 

Эру Альтрона” — очень легко. “ Войну бесконечности” — вполне возможно. По-хорошему, у всех трёх фильмов были огрехи. По-настоящему отличных тут наберётся с пяток. Но “Финал” рассчитан на всех. Посмотрел “Мстителей”. Не такая эмоциональная, не такая красивая и весёлая. Но это тоже круто.
****************************************************************************************************

Граф со степенью сжатия 20% и заданным порогом 0.2 для 3-го текста:
****************************************************************************************************
В оригинале фильм называется “Эндгейм” — это и отсылка к фразе, произнесённой Тони Старком несколько фильмов назад, и хорошее обозначение значения фильма для киновселенной: эндшпиль — шахматный термин, обозначающий переход в финальную стадию партии, где одинаковую важность приобретают и пешки, и ферзь. Потому что это на самом деле финал. Комиксфильмы в целом и киновселенную Marvel в частности принято ругать за излишнее ребячество, зачаст

После просмотра “Финала” раздражение из-за русского дубляжа рассыпается в прах. В оригинале фильм называется “Эндгейм” — это и отсылка к фразе, произнесённой Тони Старком несколько фильмов назад, и хорошее обозначение значения фильма для киновселенной: эндшпиль — шахматный термин, обозначающий переход в финальную стадию партии, где одинаковую важность приобретают и пешки, и ферзь. Потому что это на самом деле финал. Комиксфильмы в целом и киновселенную Marvel в частности принято ругать за излишнее ребячество, зачастую размытые сюжеты и картонных злодеев — и всё это справедливо. Нет, никто не поглупел и не захотел прогулять завтра первый урок. Но в этом и заключается главный фокус — на сеансе “Финала” в детей превратились абсолютно все присутствующие в зале. Это не значит, что вам нужно быть фанбоем киновселенной, чтобы кайфануть от этого фильма. Братья Руссо сняли кино одновременно для любителей копаться в бесконечных отсылках и искать связь между фильмами, и для абсолютно рядового чел